In [1]:
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
from math import log

sys.path.insert(0,'../')
from PhicoreModules import parse_genbank, mean, median, mode, visualise
from PhicoreModules import parse_genbank, median, mean, stdev, mode
from PhicoreModules import get_features_of_type, get_features_lengths, get_gc_content, get_coding_density, get_distribution_of_stops # Przemek's functions
from PhicoreModules import get_mean_cds_length_rec_window, get_rolling_gc, get_rolling_mean_cds # George's functions
from PhicoreModules import write_df_to_artemis, non_overlapping_kmers

In [2]:
def calc_coding_density(filestub='Bc01.fasta', gbkdir = '../genbank/'):
    for file in os.listdir(gbkdir):
        if not filestub in file:
            continue
        df = get_distribution_of_stops(record, window=210, step=1)

In [3]:
def stop_entropy_per_strand(entry: Bio.SeqRecord.SeqRecord, strand: int = 1, window: int = 1000, step: int = 1, verbose: bool = False) -> list[int]:
    """
    Calculate the stops per strand of the sequence
    
    :param entry: the genbank entry
    :param strand: an int {-3, -2, -1, 1, 2, 3}. If the int is -ve we will reverse complement
    :param window: the window size to use
    :param step: the interval between windows
    :param verbose: more output (to stderr)
    :return: a list with the frequency per window
    """
    
    if strand > 3 or strand < -3 or strand == 0:
        print(f"Strand must be an int of -3, -2, -1, 1, 2, 3", file=sys.stderr)
        return None
    
    if window % 3:
        print("Please make window a multiple of 3", file=sys.stderr)
        return None
    
    s = str(record.seq).upper()
    posn = strand
    
    if strand < 0:
        s = str(record.seq.reverse_complement().upper())
        posn  = -strand
    
    print(f"Start of {strand} is {s[posn:10]}", file=sys.stderr)
    result = {}
    while (posn+window+step) < len(s):
        w = s[posn:posn+window]
        entropy = 0
        
        kmers =  non_overlapping_kmers(w, 3)
        for n in [kmers.count("TAG"), kmers.count("TAA"), kmers.count("TGA")]:
            pri = n / ( (window/3) * (1/64) )
            if posn<5:
                print(f"{strand}\t{posn}\t{n}\t{pri}", file=sys.stderr)
            entropy += pri * log(pri+sys.float_info.epsilon)
        result[posn-strand] = -entropy
        posn += step
    
    return result
        
        
    

In [ ]:
file = '../genbank/Bc01.fasta-TAA.gbk'
nats = pd.DataFrame()
for record in parse_genbank(file):
    for i in [1, 2, 3, -1, -2, -3]:
        print(f"Calculating entropy for {i}", file=sys.stderr)
        data = stop_entropy_per_strand(record, strand=i, window=3996, step=1)
        if not data:
            print(f"Bugger {i}", file=sys.stderr)
        if nats.empty:
            nats = pd.DataFrame.from_dict(data, orient='index',columns=[i]).reset_index()
            nats = nats.rename(columns={'index': 'x'})
        else:
            nats[i] = nats['x'].map(data)

/home/edwa0468/.local/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1554: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       cluster_001_consensus    100102 bp    DNA             PHG\n'
  warnings.warn(
Calculating entropy for 1
Start of 1 is GCAGAAGAG
1	1	20	0.960960960960961
1	1	17	0.8168168168168168
1	1	16	0.7687687687687688
1	2	34	1.6336336336336337
1	2	69	3.315315315315315
1	2	63	3.027027027027027
1	3	11	0.5285285285285285
1	3	41	1.96996996996997
1	3	40	1.921921921921922
1	4	20	0.960960960960961
1	4	17	0.8168168168168168
1	4	16	0.7687687687687688
Calculating entropy for 2
Start of 2 is CAGAAGAG
2	2	34	1.6336336336336337
2	2	69	3.315315315315315
2	2	63	3.027027027027027
2	3	11	0.5285285285285285
2	3	41	1.96996996996997
2	3	40	1.921921921921922
2	4	20	0.960960960960961
2	4	17	0.8168168168168168
2	4	16	0.7687687687687688
Calculating entropy for 3
Start of 3 is AGAAGAG
3	3	11	0.5285285285285285
3	3	41	1.96996996996997
3	3	40	1.921921921921922
3	4	

In [ ]:
g = nats.plot(x='x', figsize=(40, 8))
t = g.set_xlabel('Position in genome (bp)')
t = g.set_ylabel('Stop codon entropy')

In [ ]:
nats